In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import math

scan_file = '/home/yan/Data2/NOMO_Project_Summary/Data/TC2_scan/ori_scan/re_name/dataset01/male/male_0024.obj'
with open(scan_file, 'r') as fp:
    content = fp.readlines()
content = [x.strip() for x in content]

targetV = []
for line in content:
    CC = line.split()
    if len(CC) > 0:
        if CC[0] == 'v':
            targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])
            
targetV = np.array(targetV)
targetV = targetV - np.mean(targetV, 0)
# minY = np.min(targetV, 0)[1]
# targetV = targetV - [0, minY, 0]

print targetV.shape

(55161, 3)


In [2]:
import pickle

# pre-process initial params
param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male/'
param_file = param_folder + 'male_0024.pkl'

with open(param_file, 'rb') as f:
    param_data = pickle.load(f)
# print param_data.shape

# init_pose = param_data[:72]
init_betas = param_data[72:]
print init_betas
 
# deg2rad = 1.0 / 180 * math.pi
# init_pose = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4, 1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
# print init_pose.shape

# init_param = np.concatenate((init_pose, init_betas))
# print init_param.shape

[ 0.50879392 -0.03695858  0.37392709 -0.04354728  0.2447124   0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]


In [3]:
## SMPL model
root_folder = '/home/yan/Data2/NOMO_Project_Summary/MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'

import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model
import numpy as np
import math

smpl_male_model = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'
outmesh_path = root_folder + 'template_male_init_pose.obj'

# load 
model = load_model(smpl_male_model)
model.betas[:] = init_betas

# zeros pose
# model.pose[:] = np.zeros(72)
# model.betas[:] = np.zeros(10)
deg2rad = 1.0 / 180 * math.pi
model.pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
model.pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])

In [4]:
# def minDist(shape_beta):
# kdtree_t = scipy.spatial.cKDTree(targetV, leafsize=6890)

def minDist(shape_pose):
    model.betas[:] = shape_pose
#     model.betas[:] = shape_pose[27:]
    print model.betas[:]
#     model.pose[48:60] = shape_pose[:12]
#     model.pose[3:18] = shape_pose[12:27]
    
    modelV = model.r
    modelV = modelV - np.mean(modelV, 0)
#     minY = np.min(modelV, 0)[1]
#     modelV = modelV - [0, minY, 0]

    # KNN searching for the nearest correspondence on Target
    # from model to target
    kdtree_t = scipy.spatial.cKDTree(targetV, leafsize=6890)
    dist_t, t_idx = kdtree_t.query(modelV, k=1)
    # from target to model
    kdtree_m = scipy.spatial.cKDTree(modelV, leafsize=6890)
    dist_m, m_idx = kdtree_m.query(targetV, k=1)
    
#     dist = (np.mean(dist_t) + np.mean(dist_m))/2 * 1000
    dist = np.mean(dist_t) * 1000
    
#     fig = plt.figure()
#     ax = Axes3D(fig)
#     ax.scatter(modelV[:,0], modelV[:, 1], modelV[:, 2], 'b.')
#     ax.scatter(targetV[t_idx,0], targetV[t_idx, 1], targetV[t_idx, 2], 'r.')
#     plt.show()
    
#     plt.figure()
#     plt.scatter(modelV[m_idx,0], modelV[m_idx, 1], c='b')
#     plt.show()
    
#     plt.figure()
#     plt.scatter(targetV[t_idx,0], targetV[t_idx, 1], c='r')
#     plt.show()
    
#     print np.mean(dist_t)*1000, np.mean(dist_m)*1000, dist
    print dist
    
    return dist

In [5]:
from scipy.optimize import minimize
import scipy

deg2rad = 1.0 / 180 * math.pi
model.pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
model.pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])

# bnds = ( (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (-90*deg2rad, -30*deg2rad), 
#         (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (30*deg2rad,   90*deg2rad),
#         (5*deg2rad,   30*deg2rad), (-30*deg2rad,-5*deg2rad), (0*deg2rad,    10*deg2rad),  
#         (5*deg2rad,   30*deg2rad), (5*deg2rad,  30*deg2rad), (-10*deg2rad,   0*deg2rad),  
#         (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (0*deg2rad,    15*deg2rad), 
#         (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (-15*deg2rad,   0*deg2rad),
#         (0*deg2rad,   10*deg2rad), (0*deg2rad,  10*deg2rad), (-5*deg2rad,    5*deg2rad),
#         (-10*deg2rad,  0*deg2rad), (-10*deg2rad, 0*deg2rad), (-10*deg2rad,   0*deg2rad), 
#         (-10*deg2rad,  0*deg2rad), (0*deg2rad,  10*deg2rad), (0*deg2rad,    10*deg2rad),)

bnds = ((-3, 3), (-3, 3), (-4, 4), (-4, 4), (-4, 4), (-4, 4), (-4, 4), (-4, 4), (-4, 4), (-4, 4))

res = minimize(minDist, init_betas, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-4, 'disp': True})
# res = minimize(minDist, init_betas, method='Nelder-Mead', options={'disp': True}) # no bnds, bad
# res = minimize(minDist, init_betas, method='Powell', options={'disp': True}) # no bnds, very bad
# res = minimize(minDist, init_betas, method='CG', options={'gtol': 1e-3, 'disp': True}) # no bnds constrains
# res = minimize(minDist, init_betas, method='BFGS', bounds=bnds, options={'gtol': 1e-3, 'disp': True}) # no bnds constrains
# res = minimize(minDist, init_betas, method='TNC', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# res = minimize(minDist, init_betas, method='SLSQP', bounds=bnds, options={'gtol': 1e-3, 'disp': True})

# res = minimize(minDist, init_betas, method='COBYLA', bounds=bnds, options={'gtol': 1e-3, 'disp': True}) # does not work
# res = minimize(minDist, init_betas, method='trust-constr', bounds=bnds, options={'gtol': 1e-3, 'disp': True}) # does not work
# res = minimize(minDist, init_betas, method='dogleg', bounds=bnds, options={'gtol': 1e-3, 'disp': True}) # Jacobian is required
# res = minimize(minDist, init_betas, method='trust-ncg', bounds=bnds, options={'gtol': 1e-3, 'disp': True}) # Jacobian is required 
# res = minimize(minDist, init_betas, method='trust-exact', bounds=bnds, options={'gtol': 1e-3, 'disp': True})  # Jacobian is required
# res = minimize(minDist, init_betas, method='trust-krylov', bounds=bnds, options={'gtol': 1e-3, 'disp': True})  # Jacobian is required

# res = minimize(minDist, init_param, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-5, 'disp': True})
# x, f, d = scipy.optimize.fmin_l_bfgs_b(minDist, init_param, bounds=bnds, pgtol=1e-5)
print res

[ 0.50879392 -0.03695858  0.37392709 -0.04354728  0.2447124   0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.519870506700652
[ 0.50879393 -0.03695858  0.37392709 -0.04354728  0.2447124   0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.519870454928135
[ 0.50879392 -0.03695857  0.37392709 -0.04354728  0.2447124   0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.519870515390284
[ 0.50879392 -0.03695858  0.3739271  -0.04354728  0.2447124   0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.519870500867825
[ 0.50879392 -0.03695858  0.37392709 -0.04354727  0.2447124   0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.519870508853458
[ 0.50879392 -0.03695858  0.37392709 -0.04354728  0.24471241  0.10404465
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.51987050133205
[ 0.50879392 -0.03695858  0.37392709 -0.04354728  0.2447124   0.10404466
 -0.07882254 -0.00305857  0.00571845  0.43246393]
31.519870510418844
[ 0.508

23.018432153631316
[ 2.09803708 -1.15465157  1.93915114 -0.8877971   1.62823693 -0.7954699
 -0.05520485  0.14126538  0.08169145  1.30616772]
23.018432159349935
[ 2.09803708 -1.15465157  1.93915114 -0.88779711  1.62823694 -0.7954699
 -0.05520485  0.14126538  0.08169145  1.30616772]
23.01843215154111
[ 2.09803708 -1.15465157  1.93915114 -0.88779711  1.62823693 -0.79546989
 -0.05520485  0.14126538  0.08169145  1.30616772]
23.018432160689205
[ 2.09803708 -1.15465157  1.93915114 -0.88779711  1.62823693 -0.7954699
 -0.05520484  0.14126538  0.08169145  1.30616772]
23.01843215695874
[ 2.09803708 -1.15465157  1.93915114 -0.88779711  1.62823693 -0.7954699
 -0.05520485  0.14126539  0.08169145  1.30616772]
23.018432155836305
[ 2.09803708 -1.15465157  1.93915114 -0.88779711  1.62823693 -0.7954699
 -0.05520485  0.14126538  0.08169146  1.30616772]
23.018432156361072
[ 2.09803708 -1.15465157  1.93915114 -0.88779711  1.62823693 -0.7954699
 -0.05520485  0.14126538  0.08169145  1.30616773]
23.01843215371

19.877669446011538
[ 2.55440933 -0.54944184  3.55691523 -2.48231833  4.         -4.
 -0.10670041  1.01956774  0.47922549  3.78643589]
19.877669446199526
[ 2.55440933 -0.54944184  3.55691523 -2.48231833  4.         -4.
 -0.10670041  1.01956774  0.47922548  3.7864359 ]
19.87766944547265
[ 2.4646654  -0.77548625  3.38164597 -2.36692934  4.         -4.
 -0.01274706  1.12352934  0.57273863  3.962849  ]
19.786786826916458
[ 2.46466541 -0.77548625  3.38164597 -2.36692934  4.         -4.
 -0.01274706  1.12352934  0.57273863  3.962849  ]
19.786786827983562
[ 2.4646654  -0.77548624  3.38164597 -2.36692934  4.         -4.
 -0.01274706  1.12352934  0.57273863  3.962849  ]
19.786786825518522
[ 2.4646654  -0.77548625  3.38164598 -2.36692934  4.         -4.
 -0.01274706  1.12352934  0.57273863  3.962849  ]
19.786786827186823
[ 2.4646654  -0.77548625  3.38164597 -2.36692933  4.         -4.
 -0.01274706  1.12352934  0.57273863  3.962849  ]
19.78678682627469
[ 2.4646654  -0.77548625  3.38164597 -2.36692

19.719424462379187
[ 2.42047742 -0.6719865   3.12324935 -1.80231824  4.         -4.
  0.2931361   1.587796    1.05051147  4.        ]
19.719424463467767
[ 2.42047742 -0.6719865   3.12324935 -1.80231825  4.00000001 -4.
  0.2931361   1.587796    1.05051147  4.        ]
19.71942446075403
[ 2.42047742 -0.6719865   3.12324935 -1.80231825  4.         -3.99999999
  0.2931361   1.587796    1.05051147  4.        ]
19.719424464954646
[ 2.42047742 -0.6719865   3.12324935 -1.80231825  4.         -4.
  0.29313611  1.587796    1.05051147  4.        ]
19.71942446330203
[ 2.42047742 -0.6719865   3.12324935 -1.80231825  4.         -4.
  0.2931361   1.58779601  1.05051147  4.        ]
19.71942446296762
[ 2.42047742 -0.6719865   3.12324935 -1.80231825  4.         -4.
  0.2931361   1.587796    1.05051148  4.        ]
19.719424462934484
[ 2.42047742 -0.6719865   3.12324935 -1.80231825  4.         -4.
  0.2931361   1.587796    1.05051147  4.00000001]
19.719424461775
[ 2.75325018 -0.63026562  3.55138084  0.5

19.643849285327445
[ 2.4530277  -0.59068063  3.66437647 -1.37641851  4.         -4.
  0.85489964  2.92344031  2.45911192  4.        ]
19.636641263025467
[ 2.45302771 -0.59068063  3.66437647 -1.37641851  4.         -4.
  0.85489964  2.92344031  2.45911192  4.        ]
19.636641261231013
[ 2.4530277  -0.59068062  3.66437647 -1.37641851  4.         -4.
  0.85489964  2.92344031  2.45911192  4.        ]
19.63664126305105
[ 2.4530277  -0.59068063  3.66437648 -1.37641851  4.         -4.
  0.85489964  2.92344031  2.45911192  4.        ]
19.63664126351465
[ 2.4530277  -0.59068063  3.66437647 -1.3764185   4.         -4.
  0.85489964  2.92344031  2.45911192  4.        ]
19.636641263310718
[ 2.4530277  -0.59068063  3.66437647 -1.37641851  4.00000001 -4.
  0.85489964  2.92344031  2.45911192  4.        ]
19.63664126026495
[ 2.4530277  -0.59068063  3.66437647 -1.37641851  4.         -3.99999999
  0.85489964  2.92344031  2.45911192  4.        ]
19.63664126505862
[ 2.4530277  -0.59068063  3.66437647 -1

...

19.615683729171177
[ 2.46037432 -0.5880836   3.52927674 -1.59760745  4.         -4.
  0.50199283  3.24767909  2.93579158  4.        ]
19.61568372951709
[ 2.46037431 -0.58808359  3.52927674 -1.59760745  4.         -4.
  0.50199283  3.24767909  2.93579158  4.        ]
19.61568372916288
[ 2.46037431 -0.5880836   3.52927675 -1.59760745  4.         -4.
  0.50199283  3.24767909  2.93579158  4.        ]
19.61568372917033
[ 2.46037431 -0.5880836   3.52927674 -1.59760744  4.         -4.
  0.50199283  3.24767909  2.93579158  4.        ]
19.615683729187403
[ 2.46037431 -0.5880836   3.52927674 -1.59760745  4.00000001 -4.
  0.50199283  3.24767909  2.93579158  4.        ]
19.61568372669896
[ 2.46037431 -0.5880836   3.52927674 -1.59760745  4.         -3.99999999
  0.50199283  3.24767909  2.93579158  4.        ]
19.615683730943097
[ 2.46037431 -0.5880836   3.52927674 -1.59760745  4.         -4.
  0.50199284  3.24767909  2.93579158  4.        ]
19.61568372917588
[ 2.46037431 -0.5880836   3.52927674 -1.

19.61563687213394
[ 2.45866714 -0.58797535  3.53257934 -1.60967384  4.         -4.
  0.49375541  3.26946713  2.96099281  4.        ]
19.615636872122547
[ 2.45866714 -0.58797535  3.53257934 -1.60967384  4.         -4.
  0.49375541  3.26946712  2.96099282  4.        ]
19.61563687211126
[ 2.45866714 -0.58797535  3.53257934 -1.60967384  4.         -4.
  0.49375541  3.26946712  2.96099281  4.00000001]
19.615636870281605
[ 2.45906971 -0.58808141  3.53051342 -1.60873534  4.         -4.
  0.48345066  3.27628674  2.96839152  4.        ]
19.61560504373353
[ 2.45906972 -0.58808141  3.53051342 -1.60873534  4.         -4.
  0.48345066  3.27628674  2.96839152  4.        ]
19.615605043849403
[ 2.45906971 -0.5880814   3.53051342 -1.60873534  4.         -4.
  0.48345066  3.27628674  2.96839152  4.        ]
19.61560504370557
[ 2.45906971 -0.58808141  3.53051343 -1.60873534  4.         -4.
  0.48345066  3.27628674  2.96839152  4.        ]
19.615605043789845
[ 2.45906971 -0.58808141  3.53051342 -1.6087353

19.615557422843864
[ 2.45797997 -0.58904373  3.52164106 -1.61179784  4.         -4.
  0.4532888   3.30477525  3.0011937   4.        ]
19.615557422878926
[ 2.45797997 -0.58904373  3.52164105 -1.61179783  4.         -4.
  0.4532888   3.30477525  3.0011937   4.        ]
19.6155574228957
[ 2.45797997 -0.58904373  3.52164105 -1.61179784  4.00000001 -4.
  0.4532888   3.30477525  3.0011937   4.        ]
19.615557420432474
[ 2.45797997 -0.58904373  3.52164105 -1.61179784  4.         -3.99999999
  0.4532888   3.30477525  3.0011937   4.        ]
19.615557424647466
[ 2.45797997 -0.58904373  3.52164105 -1.61179784  4.         -4.
  0.45328881  3.30477525  3.0011937   4.        ]
19.615557422891733
[ 2.45797997 -0.58904373  3.52164105 -1.61179784  4.         -4.
  0.4532888   3.30477526  3.0011937   4.        ]
19.61555742289571
[ 2.45797997 -0.58904373  3.52164105 -1.61179784  4.         -4.
  0.4532888   3.30477525  3.00119371  4.        ]
19.615557422891474
[ 2.45797997 -0.58904373  3.52164105 -

19.61555207970543
[ 2.45745075 -0.5885618   3.52168846 -1.61226774  4.         -4.
  0.45389261  3.30440173  3.00883188  4.        ]
19.615552079703097
[ 2.45745075 -0.5885618   3.52168846 -1.61226774  4.         -4.
  0.45389261  3.30440173  3.00883187  4.00000001]
19.615552077850392
[ 2.4574706  -0.58847028  3.52175162 -1.6122833   4.         -4.
  0.45405415  3.30411256  3.00845471  4.        ]
19.615551902059725
[ 2.45747061 -0.58847028  3.52175162 -1.6122833   4.         -4.
  0.45405415  3.30411256  3.00845471  4.        ]
19.61555190205474
[ 2.4574706  -0.58847027  3.52175162 -1.6122833   4.         -4.
  0.45405415  3.30411256  3.00845471  4.        ]
19.615551902045112
[ 2.4574706  -0.58847028  3.52175163 -1.6122833   4.         -4.
  0.45405415  3.30411256  3.00845471  4.        ]
19.6155519020628
[ 2.4574706  -0.58847028  3.52175162 -1.61228329  4.         -4.
  0.45405415  3.30411256  3.00845471  4.        ]
19.615551902062517
[ 2.4574706  -0.58847028  3.52175162 -1.6122833

19.615551575593756
[ 2.45745227 -0.58819483  3.52167562 -1.61240879  4.00000001 -4.
  0.45406682  3.30314457  3.00830306  4.        ]
19.61555157313392
[ 2.45745227 -0.58819483  3.52167562 -1.61240879  4.         -3.99999999
  0.45406682  3.30314457  3.00830306  4.        ]
19.615551577354836
[ 2.45745227 -0.58819483  3.52167562 -1.61240879  4.         -4.
  0.45406683  3.30314457  3.00830306  4.        ]
19.615551575593948
[ 2.45745227 -0.58819483  3.52167562 -1.61240879  4.         -4.
  0.45406682  3.30314458  3.00830306  4.        ]
19.615551575594065
[ 2.45745227 -0.58819483  3.52167562 -1.61240879  4.         -4.
  0.45406682  3.30314457  3.00830307  4.        ]
19.615551575593642
[ 2.45745227 -0.58819483  3.52167562 -1.61240879  4.         -4.
  0.45406682  3.30314457  3.00830306  4.00000001]
19.615551573740404
[ 2.45747532 -0.58822652  3.52163143 -1.61238555  4.         -4.
  0.4542748   3.30297587  3.0089666   4.        ]
19.615551607570982
[ 2.45747533 -0.58822652  3.52163143

In [6]:
# optimal_beta = res.x[72:]
# optimal_pose = res.x[:72]
optimal_beta = res.x
print optimal_beta

[ 2.4574592  -0.58820435  3.52166235 -1.61240181  4.         -4.
  0.45412932  3.30309387  3.00850244  4.        ]


In [10]:
model.pose[:] = np.zeros(72)
model.betas[:] = np.zeros(10)

deg2rad = 1.0 / 180 * math.pi
model.pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
model.pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
# model.pose[:] = optimal_pose
model.betas[:] = optimal_beta

modelV = model.r - np.mean(model.r, 0)
# save
with open( 'L-BFGS-B-optimized_mesh2.obj', 'w') as fp:
    for v in modelV: # [m]
        fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
    for f in model.f+1: # Faces are 1-based, not 0-based in obj files
        fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
        
print 'done'

done


In [8]:
# # save
# with open( outmesh_path, 'w') as fp:
#     for v in model.r: # [m]
#         fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
#     for f in model.f+1: # Faces are 1-based, not 0-based in obj files
#         fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
        
# print 'done'

In [9]:
# T0 = model.r
# F = model.f + 1 # Faces are 1-based, not 0-based in obj files

# # initial pose 
# theta = model.pose
# deg2rad = 1.0 / 180 * math.pi
# theta[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
# theta[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])

# J = model.J_regressor # J, the regression matrix, to estimate the joint locations, 24*6890
# W = model.weights     # W, the blend weight matrix , representing how much the rotation of part k effects the vertex i

# Sn = model.shapedirs  # Sn represents orthonormal principle components of shape displacements, 6890*3*10
# Bp = model.posedirs   # pose displacements, 6890*3*207 
# # G, a function of theta and J(beta)